In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mplsoccer import Pitch, Sbopen, VerticalPitch

In [59]:
# get Euros 2024 games
parser = Sbopen()
df_games = parser.match(competition_id=55, season_id=282)

# create empty dribbles dataframe
columns = ['match_id', 'period', 'timestamp', 'minute', 'second', 'possession', 'duration', 'match_id', 'type_name', 'outcome_name', 'team_name', 'player_name', 'x', 'y']
df_dribbles = pd.DataFrame(columns=columns)

# loop through matches and collect dribbling events
for match_id in df_games['match_id']:
    df, related, freeze, tactics = parser.event(match_id)
    dribble_events = df[df['type_name'] == 'Dribble']

    df_dribbles = pd.concat([df_dribbles, dribble_events[columns]], ignore_index=True)

df_dribbles.info()

/var/folders/5d/_v0r0r510wncd2w765n36ksw0000gn/T/ipykernel_48608/764343096.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_dribbles = pd.concat([df_dribbles, dribble_events[columns]], ignore_index=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1280 entries, 0 to 1279
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   match_id      1280 non-null   object 
 1   period        1280 non-null   object 
 2   timestamp     1280 non-null   object 
 3   minute        1280 non-null   object 
 4   second        1280 non-null   object 
 5   possession    1280 non-null   object 
 6   duration      1280 non-null   float64
 7   match_id      1280 non-null   object 
 8   type_name     1280 non-null   object 
 9   outcome_name  1280 non-null   object 
 10  team_name     1280 non-null   object 
 11  player_name   1280 non-null   object 
 12  x             1280 non-null   float64
 13  y             1280 non-null   float64
dtypes: float64(3), object(11)
memory usage: 140.1+ KB


In [61]:
df_dribbles.groupby('player_name').size().sort_values(ascending=False)

player_name
Nicholas Williams Arthuer      32
Kylian Mbappé Lottin           28
Khvicha Kvaratskhelia          28
Lamine Yamal Nasraoui Ebana    27
Jeremy Doku                    27
                               ..
László Kleinheisler             1
Marc Guehi                      1
Marin Pongracic                 1
Mario Mitaj                     1
Lazar Samardžić                 1
Length: 305, dtype: int64